# Single Path One-Shot Neural Architecture Search using Random Search

# Search the best architecture

## Setup

### Imports

In [1]:
import os
import sys
from typing import Dict

import numpy as np
import torch
import torchvision
from thop import profile
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_val_transform
from supernet import supernet18

### Make everything a bit faster

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Build datasets and dataloaders for CIFAR-10

In [3]:
# Change this value if needed.
batch_size = 512

In [4]:
val_transform = get_val_transform()

test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

Files already downloaded and verified


## Load trained supernet

In [5]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')

In [6]:
channel_multipliers = (0.25, 0.5, 0.75, 1.0, 1.25, 1.5)
supernet = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
supernet.to(device=device)

Supernet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SearchBlock(
      (ops): ModuleList(
        (0): BasicBlock(
          (conv1): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05,

In [7]:
checkpoint = torch.load('supernet_advanced_360.pth', map_location=device)
supernet.load_state_dict(checkpoint)

<All keys matched successfully>

## Apply Random Search

In [8]:
# Utility class to disable unwanted logs.

class suppress_print:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

### Define our small brute-forcer

In [9]:
@torch.no_grad()
def random_search(
        trained_supernet: nn.Module,
        val_dataloader: DataLoader,
        device: torch.device,
        n_architectures_to_test: int,
) -> Dict[str, list]:
    criterion = nn.CrossEntropyLoss()
    model = trained_supernet
    model.eval()

    unique_archs = {(None,)}
    results = {'accuracy': [], 'loss': [], 'macs': [], 'params': [], 'arch': []}
    for i in tqdm(range(n_architectures_to_test)):
        arch = [None]
        while tuple(arch) in unique_archs:
            arch = [np.random.randint(6) for i in range(8)]
        unique_archs.add(tuple(arch))

        model.sample(arch)

        with suppress_print():  # Get rid of nasty logs from thop.
            macs, params = profile(model, inputs=(torch.zeros(1, 3, 32, 32, device=device),))
        results['arch'].append(arch)
        results['macs'].append(macs)
        results['params'].append(params)

        total_loss = 0.0
        total_correct = 0.0
        total_samples = 0
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)

            logits = model(inputs)
            loss = criterion(logits, labels) * labels.shape[0]
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        accuracy = total_correct / total_samples
        loss = total_loss / total_samples
        results['accuracy'].append(accuracy)
        results['loss'].append(loss)
        print(
            f'{arch}: '
            f'MACs={(macs / 1e6):.2f}M, '
            f'params={params / 1e6:.2f}M, '
            f'accuracy={accuracy:.2%}, '
            f'loss={loss:.4f}'
        )

    return results

### Run random model evaluation

In [10]:
n_architectures_to_evaluate = 1000

random_search_results = random_search(
    supernet,
    test_dataloader,
    device,
    n_architectures_to_evaluate,
)

  0%|▏                                                                                                                                    | 1/1000 [00:01<21:43,  1.30s/it]

[0, 3, 4, 3, 5, 3, 5, 5]: MACs=40.46M, params=15.75M, accuracy=83.79%, loss=0.4735


  0%|▎                                                                                                                                    | 2/1000 [00:01<14:05,  1.18it/s]

[0, 1, 1, 3, 0, 0, 3, 3]: MACs=23.33M, params=9.43M, accuracy=82.50%, loss=0.5159


  0%|▍                                                                                                                                    | 3/1000 [00:02<11:45,  1.41it/s]

...

[0, 0, 0, 3, 2, 4, 4, 2]: MACs=27.46M, params=10.68M, accuracy=80.77%, loss=0.5634


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 999/1000 [07:13<00:00,  2.25it/s]

[3, 4, 3, 1, 5, 2, 5, 0]: MACs=34.86M, params=9.43M, accuracy=85.02%, loss=0.4518


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:13<00:00,  2.30it/s]

[0, 2, 2, 0, 2, 3, 3, 1]: MACs=24.81M, params=8.25M, accuracy=82.68%, loss=0.5036


### Save results for further analysis

In [11]:
import json
with open('random_search_1000_models_supernet_advanced_360.json', 'w') as f:
    json.dump(random_search_results, f)